<a href="https://colab.research.google.com/github/dawit-til/texts-Summarize-using-LLM-/blob/main/TEXT_SUMMERIZETION_GROUP_PROJECT_(1)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install all key libraries needed for modern NLP text summarization
!pip install transformers datasets evaluate rouge_score torch accelerate --quiet
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --ClearOutputPreprocessor.enabled=True --to notebook --inplace your_notebook.ipynb


[NbConvertApp] WARNING | pattern 'your_notebook.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
 

In [ ]:

# 🧠 Import the PyTorch library — used for deep learning and GPU computation
import torch

# ⚙️ Check if a CUDA-enabled GPU is available in the current environment
if torch.cuda.is_available():
    # ✅ If yes, print the name of the detected GPU device
    print("✅ GPU detected:", torch.cuda.get_device_name(0))
else:
    # ❌ If no GPU is found, show instructions for enabling it in Google Colab
    print("❌ No GPU detected — go to Runtime → Change runtime type → select GPU")


✅ GPU detected: Tesla T4


In [ ]:
# 🧠 Import the necessary classes from the Hugging Face Transformers library
# AutoTokenizer → automatically loads the correct tokenizer for the model
# AutoModelForSeq2SeqLM → automatically loads a sequence-to-sequence (encoder-decoder) model for text generation tasks
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ⚙️ Import PyTorch to handle tensors, GPU acceleration, and device management
import torch


# ------------------------------------------------------
# 1️⃣  DEVICE CONFIGURATION
# ------------------------------------------------------

# Select the device to run the model on: GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print out which device is being used (helps confirm GPU setup)
print(f"Using device: {device}")


# ------------------------------------------------------
# 2️⃣  MODEL AND TOKENIZER LOADING
# ------------------------------------------------------

# Define the pretrained model name from Hugging Face Hub
# "facebook/bart-large-cnn" is a BART model fine-tuned for text summarization
MODEL_NAME = "facebook/bart-large-cnn"

# Load the corresponding tokenizer — converts text to token IDs and back
# `use_fast=True` enables the optimized Rust-based tokenizer for better speed
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# Load the actual BART model for sequence-to-sequence tasks (like summarization)
# .half() → converts model weights to half-precision (float16) to save GPU memory
# .to(device) → moves the model to GPU or CPU
# .eval() → sets the model to evaluation mode (disables dropout, etc.)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).half().to(device).eval()


# ------------------------------------------------------
# 3️⃣  MODEL LENGTH CONFIGURATION
# ------------------------------------------------------

# Get the maximum sequence length (number of tokens) the model can handle
MAX_MODEL_LEN = model.config.max_position_embeddings

# Define a slightly smaller input length to leave room for special tokens
# (-3) ensures the input text doesn’t exceed the model’s max capacity
MAX_INPUT_LEN = MAX_MODEL_LEN - 3


Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
# 🧠 Disable gradient calculation during inference
# This decorator tells PyTorch not to compute or store gradients,
# which makes inference (prediction) faster and uses less memory.
@torch.inference_mode()
def summarize_gpu(texts, max_len=120, min_len=40):
    """
    Summarize input text(s) using the pretrained BART model on GPU.

    Args:
        texts (list[str] or str): One or more input texts to summarize.
        max_len (int): Maximum length of the generated summary.
        min_len (int): Minimum length of the generated summary.

    Returns:
        list[str]: The generated summaries for each input text.
    """

    # ------------------------------------------------------
    # 1️⃣ TOKENIZATION
    # ------------------------------------------------------

    # Convert raw text(s) into token IDs the model can understand
    # - padding=True → makes all sequences the same length
    # - truncation=True → cuts off texts longer than MAX_INPUT_LEN
    # - return_tensors="pt" → returns PyTorch tensors instead of lists
    enc = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=MAX_INPUT_LEN,
        return_tensors="pt"
    )


    # ------------------------------------------------------
    # 2️⃣ MOVE TO GPU AND ENSURE LENGTH LIMIT
    # ------------------------------------------------------

    # Move all tensors (input_ids, attention_mask, etc.) to the selected device (GPU or CPU)
    # Also ensure that inputs do not exceed MAX_INPUT_LEN tokens
    enc = {k: v[:, :MAX_INPUT_LEN].to(device) for k, v in enc.items()}


    # ------------------------------------------------------
    # 3️⃣ SUMMARY GENERATION
    # ------------------------------------------------------

    # Generate summaries using the model
    # Key parameters:
    # - max_length / min_length → control output summary size
    # - no_repeat_ngram_size=3 → prevents repeating 3-word sequences
    # - early_stopping=True → stops generation when an end token appears
    gen_ids = model.generate(
        **enc,
        max_length=max_len,
        min_length=min_len,
        no_repeat_ngram_size=3,
        early_stopping=True
    )


    # ------------------------------------------------------
    # 4️⃣ DECODE OUTPUT TOKENS
    # ------------------------------------------------------

    # Convert generated token IDs back to human-readable text
    # skip_special_tokens=True removes tokens like <s> or </s>
    return tokenizer.batch_decode(gen_ids, skip_special_tokens=True)


In [ ]:
# — 8. QUICK PARAGRAPH SUMMARISER ————————————————
# (Requires summarize_gpu() from Step 4 and the model/tokenizer already loaded)

paragraph = """
Football is one of the most popularly played games around the world. It is the most interesting and engaging as well as one of the oldest games in the world. This game requires a lot of enthusiasm and excitement, which is seen during tournaments and world cups in both players and spectators. It is a game of eleven players in a team where two teams play against each other. The game is named football because it is played by using their feet to pass the ball, and the use of hands is considered a foul.

Paragraph about Football in 150 Words
Among various popular games across the world, football is one of the most enthusiastic and energetic sports. This game is played between two teams, and each team has eleven members. A lot of football tournaments are held in various countries every year at the national and international levels. Every four years, the FIFA or Federation Internationale de Football Association conducts a world cup which is the most awaited championship. The game is called football because the ball is passed among the team members using their feet, and the use of the hand is considered a foul. This, however, is decided by the referee of the match. The International Football Association Board formulates rules that the players follow, and any contravention shall be considered a disqualification.
"""

summary = summarize_gpu([paragraph], max_len=80, min_len=35)[0]
print("- Original ({} chars) -\n\n{}".format(len(paragraph), paragraph))
print("- Summary -\n{}".format(summary))


- Original (1326 chars) -


Football is one of the most popularly played games around the world. It is the most interesting and engaging as well as one of the oldest games in the world. This game requires a lot of enthusiasm and excitement, which is seen during tournaments and world cups in both players and spectators. It is a game of eleven players in a team where two teams play against each other. The game is named football because it is played by using their feet to pass the ball, and the use of hands is considered a foul.

Paragraph about Football in 150 Words
Among various popular games across the world, football is one of the most enthusiastic and energetic sports. This game is played between two teams, and each team has eleven members. A lot of football tournaments are held in various countries every year at the national and international levels. Every four years, the FIFA or Federation Internationale de Football Association conducts a world cup which is the most awaited champio